# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfc_recovered = dft_recovered.groupby(COL_REGION)[dft_recovered.columns[-1]].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()
dfp_recovered = dft_recovered.groupby(COL_REGION)[dft_recovered.columns[-6]].sum()

In [16]:
#hide
dfc_recovered = dft_cases.copy()
dfc_recovered.loc[:, dft_cases.columns[4:]] = 0
dfc_recovered

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,21,22,22,22,24,24,40,40,74,84
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,51,55,59,64,70,76,89,104,123,146
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,54,60,74,87,90,139,201,230,264,302
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,2,39,39,53,75,88,113,133,164,188
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,0,0,0,0,1,2,2,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,NaN,Libya,26.335100,17.228331,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
241,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,38,39,41,44,47,48,52,59,59,-1
242,NaN,Guinea-Bissau,11.803700,-15.180400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
243,NaN,Mali,17.570692,-3.996166,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [5]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths, Recovered'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate,Continent
0,China,81661.0,3285.0,73280.0,81250.0,3253.0,70535.0,411.0,32.0,2745.0,4.0,Asia
1,Italy,74386.0,7503.0,8326.0,47021.0,4032.0,4440.0,27365.0,3471.0,3886.0,10.1,Europe
2,US,65778.0,942.0,348.0,19100.0,244.0,121.0,46678.0,698.0,227.0,1.4,North America
3,Spain,49515.0,3647.0,3794.0,20410.0,1043.0,1107.0,29105.0,2604.0,2687.0,7.4,Europe
4,Germany,37323.0,206.0,3243.0,19848.0,67.0,113.0,17475.0,139.0,3130.0,0.6,Europe
5,Iran,27017.0,2077.0,8913.0,19644.0,1433.0,5710.0,7373.0,644.0,3203.0,7.7,Asia
6,France,25600.0,1333.0,3250.0,12758.0,451.0,12.0,12842.0,882.0,3238.0,5.2,Europe
7,Switzerland,10897.0,153.0,131.0,5294.0,54.0,15.0,5603.0,99.0,116.0,1.4,Europe
8,United Kingdom,9640.0,466.0,140.0,4014.0,178.0,67.0,5626.0,288.0,73.0,4.8,Europe
9,South Korea,9137.0,126.0,3507.0,8652.0,94.0,1540.0,485.0,32.0,1967.0,1.4,Asia


In [6]:
#hide

#delete problematic countries from table
df_table = df_table[df_table['Country/Region'] != 'Cape Verde']
df_table = df_table[df_table['Country/Region'] != 'Cruise Ship']
df_table = df_table[df_table['Country/Region'] != 'Kosovo']

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2020-03-25 00:00:00'),
 'since': Timestamp('2020-03-20 00:00:00'),
 'Cases': 467593.0,
 'Deaths': 21180.0,
 'Recovered': 107659.0,
 'Cases (+)': 195608.0,
 'Deaths (+)': 9886.0,
 'Recovered (+)': 23018.0,
 'China Cases': 81661.0,
 'China Deaths': 3285.0,
 'China Recovered': 73280.0,
 'China Cases (+)': 411.0,
 'China Deaths (+)': 32.0,
 'China Recovered (+)': 2745.0,
 'US Cases': 65778.0,
 'US Deaths': 942.0,
 'US Recovered': 348.0,
 'US Cases (+)': 46678.0,
 'US Deaths (+)': 698.0,
 'US Recovered (+)': 227.0,
 'EU Cases': 248822.0,
 'EU Deaths': 14177.0,
 'EU Recovered': 19756.0,
 'EU Cases (+)': 119470.0,
 'EU Deaths (+)': 8116.0,
 'EU Recovered (+)': 13846.0}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,5,1,0,0,2,0,16,0,34,10
Albania,0,0,0,0,0,0,0,0,0,0,...,9,4,4,5,6,6,13,15,19,23
Algeria,0,0,0,0,0,0,0,0,0,0,...,6,6,14,13,3,49,62,29,34,38
Andorra,0,0,0,0,0,0,0,0,0,0,...,1,37,0,14,22,13,25,20,31,24
Angola,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,0


In [9]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)